In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from itertools import islice


In [2]:
def weights_init(m):
    for name, param in m.named_parameters(): 
        nn.init.normal_(param)
        print(name)

In [3]:
# Reading in file and adding it to array to one hot encode. 
#file = open('samplemusic.txt')
file = open('train.txt')
tag = False
part_tag = False
array = []
tag_string = ''
while 1:
    char = file.read(1)  # read by character
    if not char: break
        
    # if char is beginning of tag
    if char == '<':
        part_tag = True
        tag_string += char
        
    # if char is end of tag
    elif char == '>':
        # Checks if > is part of <start>/<end> tags or lone char
        if tag_string == '<start' or tag_string == '<end':
            tag = False
            tag_string += char
            array.append(tag_string)
        # > is not part of tag, just lone char so we append it and reset the tag_string
        else: 
            array.append(char)
        tag_string = ''
        
    
    # checks if char is still part of tag
    elif part_tag == True:
        # If next char after < is either s or e, we assume this is a tag.
        if char == 's' or char == 'e':
            tag = True
            tag_string += char
        # If not, < is just a lone char so append both separately
        else:
            array.append(tag_string)
            array.append(char)
            tag_string = ''
        part_tag = False
        
    # If inside tag, keep adding chars to create full tag
    elif tag == True:
        tag_string += char
            
    # if char is not part of tag, just add char
    elif tag == False:
        array.append(char)
    
file.close()

In [4]:
# One-hot encoding of data
array = np.array(array).reshape(-1,1)
print('Shape or original data array:', array.shape)
print('Number of unique chars in array:', len(np.unique(array)))
onehot_encoder = OneHotEncoder(sparse=False)
X_data = onehot_encoder.fit_transform(array) # Contains One hot encoding of all chars in text 

# Example of inversing one hot to get letter
inverted = onehot_encoder.inverse_transform(X_data[0, :].reshape(1,-1))
print(inverted)
print('Shape of one-hot encoded data, X_data:', X_data.shape)
num_chars = X_data.shape[0]
num_hot_encoding = X_data.shape[1]

Shape or original data array: (185410, 1)
Number of unique chars in array: 153
[['<start>']]
Shape of one-hot encoded data, X_data: (185410, 153)


In [5]:
# Turning songs from data into chunks 
train_data = []
song = []
temp = []
for idx in range(len(X_data)):
    temp.append(X_data[idx]) # Add 100 chars into temp to create chunks 
    chara = onehot_encoder.inverse_transform(X_data[idx, :].reshape(1,-1))[0][0]
    # Checks if char is at the end of a song 
    #print(chara)
    if chara == '<end>':
        song.append(np.array(temp))
        train_data.append(np.array(song))
        temp = []
        song = []
    
    # If 100 chars in temp, add chunk to train_data
    if len(temp) == 100:
        song.append(np.array(temp))
        temp = []

train_data = np.array(train_data)
print(train_data.shape[0])

333


In [6]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        #self.sequence_length = sequence_length

        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, batch_first=True)

    def forward(self, x, *args):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size) for batch_first=True
        hidden = args[0][0]
        cell = args[0][1]
        # Reshape input
        x.view(1, x.shape[1], self.input_size)
        out, (hidden, cell) = self.lstm(x, (hidden,cell))
        return out.view(-1, num_classes), (hidden,cell)
    
    def init_hidden(self):
        if torch.cuda.is_available():
            hidden = torch.zeros(self.num_layers, 1, self.hidden_size).cuda()
            cell = torch.zeros(self.num_layers, 1, self.hidden_size).cuda()
        else:
            hidden = torch.zeros(self.num_layers, 1, self.hidden_size)
            cell = torch.zeros(self.num_layers, 1, self.hidden_size)


        return (Variable(hidden), Variable(cell))

In [7]:
# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

CUDA is supported


In [25]:
torch.manual_seed(777)  # reproducibility
num_classes = num_hot_encoding
input_size = num_hot_encoding  # one-hot size
hidden_size = num_hot_encoding  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
#sequence_length = 100  # One hundred sized chunks of text 
num_layers = 1  # one-layer rnn

# Instantiate RNN model
lstm = LSTM(num_classes, input_size, hidden_size, num_layers).to(device)
lstm.apply(weights_init)
print(lstm)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.01)


# Train the model
for epoch in range(1):
    correct = 0
    total = 0
    song_count = 0
    song_loss = []
    # Loop through every song in the dataset
    for song in train_data:
        hidden_cont = None
        chunk_loss = 0.0
        #optimizer.zero_grad()
        # Loop through every chunk, songs are broken up into chunks 
        for chunk_count, chunk in enumerate(song):
            targets = np.roll(chunk, -1, axis=0) # Targets is the chunk left shifted once
            #print('chunk', chunk)
            #print('targets', targets)
            # If chunk is first one in song, init hidden state, else keep hidden state for next chunk
            if chunk_count == 0:
                #print('init')
                hidden = lstm.init_hidden() # Inits hidden state
            else:
                #print('cont')
                hidden = hidden_cont
                
            optimizer.zero_grad()
            letter = torch.Tensor(chunk).view(1,-1,153).to(device)
            target_letter = torch.LongTensor(targets).view(-1,153).to(device)
            output, hidden = lstm(letter, hidden)
            hidden_cont = (hidden[0].detach(), hidden[1].detach())
            #print('out:', torch.argmax(output, dim=1))
            #print('targ:', torch.argmax(target_letter, dim=1))
            loss = criterion(output, torch.argmax(target_letter, dim=1))
            loss.backward()
            optimizer.step()
            chunk_loss += loss.item()
            
            #trained_weights = lstm.lstm.weight_ih_l0
            #print(trained_weights)
            
        #print("Finished", song_count + 1, "/333 songs of training")
        #print('Avg chunk loss: %.3f' % (chunk_loss/chunk_count+1))
        song_loss.append(chunk_loss/(chunk_count+1))
        chunk_loss = 0.0
        song_count += 1
        
     
    avg_song_loss = np.mean(song_loss)
    print("epoch: %d, avg loss: %1.3f" % (epoch + 1, avg_song_loss))

print("Learning finished!")

weight_ih_l0
weight_hh_l0
bias_ih_l0
bias_hh_l0
lstm.weight_ih_l0
lstm.weight_hh_l0
lstm.bias_ih_l0
lstm.bias_hh_l0
LSTM(
  (lstm): LSTM(153, 153, batch_first=True)
)
Parameter containing:
tensor([[ 0.4473, -1.1195, -1.9564,  ..., -0.1344, -0.3040, -1.2460],
        [-0.1194,  0.4202, -1.5847,  ...,  0.7577, -0.1025, -0.9679],
        [ 1.1497, -1.5898,  1.1344,  ...,  0.4415, -0.1360,  0.7439],
        ...,
        [ 0.1148,  1.6949,  1.3711,  ...,  0.4732,  0.2266, -0.0565],
        [-0.8032,  0.3582, -1.7010,  ...,  0.6959,  0.2755,  0.3027],
        [ 0.7656,  0.0362, -1.4344,  ..., -0.6544,  0.7901, -0.5764]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.4473, -1.1262, -1.9564,  ..., -0.1270, -0.3114, -1.2460],
        [-0.1194,  0.4269, -1.5847,  ...,  0.7652, -0.0950, -0.9679],
        [ 1.1497, -1.5965,  1.1344,  ...,  0.4489, -0.1434,  0.7439],
        ...,
        [ 0.1148,  1.7016,  1.3711,  ...,  0.4807,  0.2340, -0.0565],
        [-0.8032, 

Parameter containing:
tensor([[ 0.4473, -1.1557, -1.9564,  ..., -0.0883, -0.2706, -1.2460],
        [-0.1194,  0.4361, -1.5847,  ...,  0.8099, -0.1359, -0.9679],
        [ 1.1497, -1.6310,  1.1344,  ...,  0.4781, -0.1026,  0.7439],
        ...,
        [ 0.1148,  1.7311,  1.3711,  ...,  0.5339,  0.1932, -0.0565],
        [-0.8032,  0.3917, -1.7010,  ...,  0.6503,  0.2421,  0.3027],
        [ 0.7656, -0.0036, -1.4344,  ..., -0.7150,  0.7418, -0.5764]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.4473, -1.1564, -1.9564,  ..., -0.0866, -0.2694, -1.2460],
        [-0.1194,  0.4358, -1.5847,  ...,  0.8116, -0.1370, -0.9679],
        [ 1.1497, -1.6320,  1.1344,  ...,  0.4798, -0.1014,  0.7439],
        ...,
        [ 0.1148,  1.7318,  1.3711,  ...,  0.5356,  0.1921, -0.0565],
        [-0.8032,  0.3923, -1.7010,  ...,  0.6486,  0.2410,  0.3027],
        [ 0.7656, -0.0046, -1.4344,  ..., -0.7167,  0.7406, -0.5764]],
       device='cuda:0', requires_grad=True)


Parameter containing:
tensor([[ 0.4473, -1.1652, -1.9564,  ..., -0.1056, -0.2586, -1.2460],
        [-0.1194,  0.4330, -1.5847,  ...,  0.7957, -0.1722, -0.9679],
        [ 1.1497, -1.5927,  1.1344,  ...,  0.4960, -0.0903,  0.7439],
        ...,
        [ 0.1148,  1.7234,  1.3711,  ...,  0.5471,  0.1812, -0.0565],
        [-0.8032,  0.4014, -1.7010,  ...,  0.6369,  0.2301,  0.3027],
        [ 0.7656, -0.0140, -1.4344,  ..., -0.6810,  0.7298, -0.5764]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.4473, -1.1653, -1.9564,  ..., -0.1121, -0.2585, -1.2460],
        [-0.1194,  0.4330, -1.5847,  ...,  0.7903, -0.1758, -0.9679],
        [ 1.1497, -1.5916,  1.1344,  ...,  0.5030, -0.0903,  0.7439],
        ...,
        [ 0.1148,  1.7231,  1.3711,  ...,  0.5540,  0.1812, -0.0565],
        [-0.8032,  0.4014, -1.7010,  ...,  0.6321,  0.2300,  0.3027],
        [ 0.7656, -0.0145, -1.4344,  ..., -0.6879,  0.7298, -0.5764]],
       device='cuda:0', requires_grad=True)


Parameter containing:
tensor([[ 0.4473, -1.1659, -1.9564,  ..., -0.1222, -0.2581, -1.2460],
        [-0.1194,  0.4330, -1.5847,  ...,  0.7527, -0.2096, -0.9679],
        [ 1.1497, -1.5814,  1.1344,  ...,  0.5677, -0.0894,  0.7439],
        ...,
        [ 0.1148,  1.7202,  1.3711,  ...,  0.6181,  0.1807, -0.0565],
        [-0.8032,  0.4022, -1.7010,  ...,  0.5882,  0.2295,  0.3027],
        [ 0.7656, -0.0184, -1.4344,  ..., -0.7583,  0.7293, -0.5764]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.4473, -1.1659, -1.9564,  ..., -0.1204, -0.2581, -1.2460],
        [-0.1194,  0.4330, -1.5847,  ...,  0.7527, -0.2098, -0.9679],
        [ 1.1497, -1.5814,  1.1344,  ...,  0.5680, -0.0894,  0.7439],
        ...,
        [ 0.1148,  1.7202,  1.3711,  ...,  0.6185,  0.1807, -0.0565],
        [-0.8032,  0.4022, -1.7010,  ...,  0.5879,  0.2295,  0.3027],
        [ 0.7656, -0.0185, -1.4344,  ..., -0.7589,  0.7293, -0.5764]],
       device='cuda:0', requires_grad=True)


Parameter containing:
tensor([[ 0.4473, -1.1663, -1.9564,  ..., -0.1046, -0.2581, -1.2460],
        [-0.1194,  0.4331, -1.5847,  ...,  0.7315, -0.2116, -0.9679],
        [ 1.1497, -1.5808,  1.1344,  ...,  0.5721, -0.0894,  0.7439],
        ...,
        [ 0.1148,  1.7205,  1.3711,  ...,  0.6244,  0.1807, -0.0565],
        [-0.8032,  0.4021, -1.7010,  ...,  0.5200,  0.2295,  0.3027],
        [ 0.7656, -0.0189, -1.4344,  ..., -0.7615,  0.7293, -0.5764]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.4473, -1.1663, -1.9564,  ..., -0.1045, -0.2581, -1.2460],
        [-0.1194,  0.4331, -1.5847,  ...,  0.7309, -0.2116, -0.9679],
        [ 1.1497, -1.5808,  1.1344,  ...,  0.5721, -0.0894,  0.7439],
        ...,
        [ 0.1148,  1.7205,  1.3711,  ...,  0.6245,  0.1807, -0.0565],
        [-0.8032,  0.4021, -1.7010,  ...,  0.5193,  0.2295,  0.3027],
        [ 0.7656, -0.0189, -1.4344,  ..., -0.7629,  0.7293, -0.5764]],
       device='cuda:0', requires_grad=True)


...

Parameter containing:
tensor([[ 0.7128, -1.2380, -1.9564,  ..., -0.2184, -0.0957, -1.5366],
        [ 0.1461,  0.2466, -1.5847,  ...,  0.5514, -0.2801, -0.8176],
        [ 0.8866, -1.8574,  1.1344,  ...,  0.5341, -0.0284,  1.5063],
        ...,
        [ 0.3803,  1.7035,  1.3711,  ...,  0.3935, -0.0183,  0.9212],
        [-1.0686,  0.1368, -1.7010,  ...,  0.7623, -0.0243,  0.5205],
        [ 1.0310, -0.2726, -1.4344,  ..., -0.4597,  0.6009, -1.5321]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.7128, -1.2380, -1.9564,  ..., -0.2184, -0.0957, -1.5366],
        [ 0.1461,  0.2463, -1.5847,  ...,  0.5514, -0.2801, -0.8176],
        [ 0.8866, -1.8575,  1.1344,  ...,  0.5341, -0.0284,  1.5063],
        ...,
        [ 0.3803,  1.7035,  1.3711,  ...,  0.3935, -0.0183,  0.9212],
        [-1.0686,  0.1368, -1.7010,  ...,  0.7623, -0.0243,  0.5205],
        [ 1.0310, -0.2726, -1.4344,  ..., -0.4597,  0.6009, -1.5321]],
       device='cuda:0', requires_grad=True)


Parameter containing:
tensor([[ 0.7128, -1.2379, -1.9564,  ..., -0.2184, -0.0957, -1.5366],
        [ 0.1461,  0.2441, -1.5847,  ...,  0.5514, -0.2801, -0.8176],
        [ 0.8866, -1.8576,  1.1344,  ...,  0.5341, -0.0284,  1.5063],
        ...,
        [ 0.3803,  1.7055,  1.3711,  ...,  0.3935, -0.0183,  0.9212],
        [-1.0686,  0.1368, -1.7010,  ...,  0.7623, -0.0243,  0.5205],
        [ 1.0310, -0.2733, -1.4344,  ..., -0.4598,  0.6009, -1.5321]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.7128, -1.2379, -1.9564,  ..., -0.2184, -0.0957, -1.5366],
        [ 0.1461,  0.2441, -1.5847,  ...,  0.5514, -0.2801, -0.8176],
        [ 0.8866, -1.8576,  1.1344,  ...,  0.5341, -0.0284,  1.5063],
        ...,
        [ 0.3803,  1.7055,  1.3711,  ...,  0.3935, -0.0183,  0.9212],
        [-1.0686,  0.1368, -1.7010,  ...,  0.7623, -0.0243,  0.5205],
        [ 1.0310, -0.2733, -1.4344,  ..., -0.4598,  0.6009, -1.5321]],
       device='cuda:0', requires_grad=True)


Parameter containing:
tensor([[ 0.7128, -1.2412, -1.9564,  ..., -0.2184, -0.0957, -1.5366],
        [ 0.1461,  0.2437, -1.5847,  ...,  0.5514, -0.2801, -0.8176],
        [ 0.8866, -1.8572,  1.1344,  ...,  0.5341, -0.0284,  1.5063],
        ...,
        [ 0.3803,  1.7073,  1.3711,  ...,  0.3935, -0.0183,  0.9212],
        [-1.0686,  0.1362, -1.7010,  ...,  0.7623, -0.0243,  0.5205],
        [ 1.0310, -0.2734, -1.4344,  ..., -0.4598,  0.6009, -1.5321]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.7128, -1.2416, -1.9564,  ..., -0.2184, -0.0957, -1.5366],
        [ 0.1461,  0.2437, -1.5847,  ...,  0.5514, -0.2801, -0.8176],
        [ 0.8866, -1.8571,  1.1344,  ...,  0.5341, -0.0284,  1.5063],
        ...,
        [ 0.3803,  1.7074,  1.3711,  ...,  0.3935, -0.0183,  0.9212],
        [-1.0686,  0.1362, -1.7010,  ...,  0.7623, -0.0243,  0.5205],
        [ 1.0310, -0.2734, -1.4344,  ..., -0.4598,  0.6009, -1.5321]],
       device='cuda:0', requires_grad=True)


Parameter containing:
tensor([[ 0.7128, -1.2440, -1.9564,  ..., -0.2183, -0.0957, -1.5366],
        [ 0.1461,  0.2437, -1.5847,  ...,  0.5514, -0.2801, -0.8176],
        [ 0.8866, -1.8566,  1.1344,  ...,  0.5333, -0.0284,  1.5063],
        ...,
        [ 0.3803,  1.7083,  1.3711,  ...,  0.3936, -0.0183,  0.9212],
        [-1.0686,  0.1360, -1.7010,  ...,  0.7623, -0.0243,  0.5205],
        [ 1.0310, -0.2734, -1.4344,  ..., -0.4596,  0.6009, -1.5321]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.7128, -1.2440, -1.9564,  ..., -0.2183, -0.0957, -1.5366],
        [ 0.1461,  0.2437, -1.5847,  ...,  0.5514, -0.2801, -0.8176],
        [ 0.8866, -1.8566,  1.1344,  ...,  0.5333, -0.0284,  1.5063],
        ...,
        [ 0.3803,  1.7083,  1.3711,  ...,  0.3936, -0.0183,  0.9212],
        [-1.0686,  0.1360, -1.7010,  ...,  0.7623, -0.0243,  0.5205],
        [ 1.0310, -0.2733, -1.4344,  ..., -0.4596,  0.6009, -1.5321]],
       device='cuda:0', requires_grad=True)


Parameter containing:
tensor([[ 0.7128, -1.2445, -1.9564,  ..., -0.2183, -0.0957, -1.5366],
        [ 0.1461,  0.2437, -1.5847,  ...,  0.5504, -0.2801, -0.8176],
        [ 0.8866, -1.8566,  1.1344,  ...,  0.5331, -0.0284,  1.5063],
        ...,
        [ 0.3803,  1.7085,  1.3711,  ...,  0.3936, -0.0183,  0.9212],
        [-1.0686,  0.1363, -1.7010,  ...,  0.7623, -0.0243,  0.5205],
        [ 1.0310, -0.2733, -1.4344,  ..., -0.4595,  0.6009, -1.5321]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.7128, -1.2445, -1.9564,  ..., -0.2183, -0.0957, -1.5366],
        [ 0.1461,  0.2437, -1.5847,  ...,  0.5503, -0.2801, -0.8176],
        [ 0.8866, -1.8566,  1.1344,  ...,  0.5331, -0.0284,  1.5063],
        ...,
        [ 0.3803,  1.7085,  1.3711,  ...,  0.3936, -0.0183,  0.9212],
        [-1.0686,  0.1363, -1.7010,  ...,  0.7623, -0.0243,  0.5205],
        [ 1.0310, -0.2733, -1.4344,  ..., -0.4595,  0.6009, -1.5321]],
       device='cuda:0', requires_grad=True)
